In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
from IPython.display import Image, display
import torch
import glob

# Fix a seed for PyTorch
torch.manual_seed(4200);

## Enable GPU

gpu = True

if gpu == True:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# Roboflow datasets are used in this notebook 
!pip install roboflow
from roboflow import Roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=915f14260477679c8c1bf375bd9afc7bcd77c64b7f840045801957847f78b708
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pypar

In [5]:
# Connect to Roboflow website using your account's workspace private API
rf = Roboflow(api_key="fGlQHNltZ7WWmQyKUtdA")

# link to the workspace of dataset wanted
project = rf.workspace("david-lee-d0rhs").project("american-sign-language-letters")

# download the dataset for yolov5
dataset = project.version(6).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to American-Sign-Language-Letters-6 in yolov5pytorch:: 100%|██████████| 1452/1452 [00:01<00:00, 1372.37it/s]


In [ ]:
# read a sample from the valid/images, for it's shape details and so on
import cv2

img = cv2.imread('/content/American-Sign-Language-Letters-6/valid/images/A10_jpg.rf.85a1d5108a4be1e892ed2d1eab546db9.jpg')
img.shape

(640, 640, 3)

In [19]:
# lets look at the data
!more /content/yolov5/American-Sign-Language-Letters-6/data.yaml

names:
- A
- B
- C
- D
- E
- F
- G
- H
- I
- J
- K
- L
- M
- N
- O
- P
- Q
- R
- S
- T
- U
- V


In [6]:
# clone the yolov5 and install all the requirements

!git clone https://github.com/ultralytics/yolov5  # clone
%cd ./yolov5
%pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16000, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 16000 (delta 104), reused 110 (delta 69), pack-reused 15831
Receiving objects: 100% (16000/16000), 14.64 MiB | 35.96 MiB/s, done.
Resolving deltas: 100% (10977/10977), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.4/612.4 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
# move the dataset inside yolov5 (can also change the path used by default in the train.py)
!mv /content/American-Sign-Language-Letters-6 /content/yolov5/American-Sign-Language-Letters-6

In [ ]:
# Track the weights and biases automatically using wandb 
# !pip install wandb
# !wandb login
# import wandb
# wandb.init(project="YOLOv5-ASL", entity="jayanth")

In [27]:
# best and last model weights are saved automatically and all the epochs results are also saved in runs/exp{number} folder of yolov5

# train the yolov5 model from scratch, add the '--save-dir /content/yolov5/checkpoints' at the end of below command to save checkpoints
# !python train.py --img 640 --batch 10 --epochs 50 --data /content/yolov5/American-Sign-Language-Letters-6/data.yaml --weights '' --cfg /content/yolov5/models/yolov5s.yaml --cache ram

# since the dataset is not that large, lets use transfer learning

# train the yolov5 model from pretrained weights, add the '--save-dir /content/yolov5/checkpoints' at the end of below command to save checkpoints 
!python train.py --img 640 --batch 10 --epochs 50 --data /content/yolov5/American-Sign-Language-Letters-6/data.yaml --weights yolov5s.pt --cache ram

train: weights=yolov5s.pt, cfg=, data=/content/yolov5/American-Sign-Language-Letters-6/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-186-g0acc5cf Python-3.10.12 torch-2.0.1+cu118 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, 

In [28]:
# Zip the YOLOv5 folder in order to download it
!zip -r /content/yolov5.zip /content/yolov5

# and download manually:
#   * 'README.robotflow.txt'
#   * 'data.yaml'
#   * 'yolov5.zip'
#   * 'yolov5s.pt'
#   * this notebook

  adding: content/yolov5/ (stored 0%)
  adding: content/yolov5/American-Sign-Language-Letters-6/ (stored 0%)
  adding: content/yolov5/American-Sign-Language-Letters-6/data.yaml (deflated 46%)
  adding: content/yolov5/American-Sign-Language-Letters-6/README.roboflow.txt (deflated 45%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/ (stored 0%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/labels.cache (deflated 68%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/labels/ (stored 0%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/labels/M5_jpg.rf.63cf5f9e88527a60b4ccbd30e55cdf19.txt (deflated 31%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/labels/R17_jpg.rf.2384dd6bb4be41cd5204499c45a36cb6.txt (deflated 25%)
  adding: content/yolov5/American-Sign-Language-Letters-6/valid/labels/S2_jpg.rf.d1deed1d4059d5d0fbf71d14a425e611.txt (deflated 27%)
  adding: content/yolov5/American-Sign-Language-Letters-6/vali

In [30]:
# from utils.plots import plot_results
# # plot 'results.csv' as 'results.png'
# plot_results('/content/yolov5/runs/train/exp10/results.csv') 
# # will already be there in the '/conent/yolov5/runs/train/exp10' as 'results.png'

In [ ]:
# !python train.py --resume

In [ ]:
# save yolov5s.pt (weights after training and also include the model yolov5s.pb)
!python /content/yolov5/export.py --weights yolov5s.pt --include torchscript pb

In [ ]:
# # include torchscript for the best model weights of all the epochs in the experiment (also weights are automatically saved in the yolov5/runs/exp folder)
# !python /content/yolov5/export.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --include torchscript

In [29]:
# copy the yolov5 folder to drive
!cp /content/yolov5 /content/drive/MyDrive/ML_Proj -r

In [ ]:
# !wandb offline

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.
